## TPS Nov. 2021 - Baseline non-boost

## Import libraries

In [ ]:
%%time

import os
import logging
import sys
import time
from datetime import timedelta

import warnings
warnings.simplefilter("ignore")

import gc
gc.enable()

import numpy as np
import pandas as pd

import plotly.figure_factory as ff

from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

## Load datasets

In [ ]:
%%time

data_dir = "../input/tabular-playground-series-nov-2021/"

train  = pd.read_csv(data_dir  + "train.csv")
test = pd.read_csv(data_dir + "test.csv")
submission = pd.read_csv(data_dir + "sample_submission.csv")

In [ ]:
%%time

print("Train shape: ", train.shape)
print("Test shape: ", test.shape, end="\n\n")

In [ ]:
train.head()

In [ ]:
test.head()

## Features

In [ ]:
TARGET = "target"

features = [col for col in train.columns if col not in ('id', TARGET)]

In [ ]:
%%time

y = train.pop(TARGET)
X = train.copy()
X_test = test.drop("id", axis=1).copy()

del train
del test

## Predict

In [ ]:
%%time

def predict_with_model(model, simple_fit=False, splits=5):
    test_preds = []
    valid_preds = {}
    scores = []
    
    skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)
    for fold, (idx_train, idx_valid) in enumerate(skf.split(X, y)):
        start_time = time.monotonic()
        
        X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
        
        valid_ids = X_valid.id.values.tolist()

        X_train = X_train[features]
        X_valid = X_valid[features]

        if simple_fit:
            model.fit(X_train, y_train)
        else:
            model.fit(
                X_train, y_train,
                eval_set=[(X_valid, y_valid)],
                early_stopping_rounds=180,
                verbose=1000
            )
        
        valid_pred = model.predict_proba(X_valid)[:, 1]
        test_pred = model.predict_proba(X_test)[:, 1]
        
        test_preds.append(test_pred)
        valid_preds.update(dict(zip(valid_ids, valid_pred)))

        score = roc_auc_score(y_valid, valid_pred)
        
        end_time = time.monotonic()
        dur = timedelta(seconds=end_time - start_time)
        print(f"Fold {fold} | AUC: {score} | Took: {dur}")
        scores.append(score)
    
    test_preds = np.mean(np.column_stack(test_preds), axis=1)
    valid_preds = pd.DataFrame.from_dict(valid_preds, orient="index").reset_index()
    
    return test_preds, valid_preds, scores

In [ ]:
%%time

def predict_with_models(models):
    print(f"Predicting with {len(models)} models...", end="\n\n")
    for model_name, model in models:
        start_time = time.monotonic()
        
        # simple fit for sklearn models
        simple_fit = True
        
        print("-" * 50)
        print(f"Using {model_name} model...")
        test_preds, valid_preds, scores = predict_with_model(model, simple_fit=simple_fit)
        print(f"Score: {np.mean(scores)}, Std: {np.std(scores)}", end="\n\n")

        print("Saving predictions...")
        valid_preds.columns = ["id", model_name]
        valid_preds.to_csv(f"{model_name}_train.csv", index=False)

        test_preds_df = pd.DataFrame({"id": submission.id, model_name: test_preds})
        test_preds_df.to_csv(f"{model_name}_test.csv", index=False)

        sub = pd.DataFrame({"id": submission.id, TARGET: test_preds})
        sub.to_csv(f"{model_name}_submission.csv", index=False)
        
        end_time = time.monotonic()
        dur = timedelta(seconds=end_time - start_time)
        print(f"Took: {dur}")

In [ ]:
%%time

SEED=42

lr1_params = {
    "solver": "liblinear"
}

sgdc1_params = {
    "tol": 1e-5,
    "loss": "log",
    "n_jobs": -1,
    "early_stopping": True,
    "random_state": SEED,
}

lsvc1_params = {
    "tol": 1e-5,
    "random_state": SEED,
}

models = [
    ("lda1", LinearDiscriminantAnalysis()),
    ("lr1", LogisticRegression(**lr1_params)),
    ("sgdc1", SGDClassifier(**sgdc1_params)),
    ("lsvc1", CalibratedClassifierCV(base_estimator=LinearSVC(**lsvc1_params))),
]

In [ ]:
%%time

predict_with_models(models)

## Visualize

In [ ]:
def load_viz_data(submission_files):
    dfs = []
    for submission_file in submission_files:
        df = pd.read_csv(submission_file)
        dfs.append((submission_file, df))
        
    hist_data = []
    for i in range(len(dfs)):
        _, df = dfs[i]
        hist_data.append(df[TARGET])
        
    return hist_data

In [ ]:
%%time

submission_files = [f"{model_name}_submission.csv" for (model_name, _) in models]
viz_data = load_viz_data(submission_files)

fig = ff.create_distplot(viz_data, submission_files, show_hist=False, show_rug=False)
fig.show()